In [ ]:
#!coding=utf-8

import threading
import socket
import struct
#import pyDes
import base64
from Crypto.Cipher import AES
import rsa
import time

#key = '6A4B3C7D9E2F1F3F'
#IV = [0x51, 0xA2, 0x6C, 0x32, 0x11, 0xF1, 0xD4, 0x09]
vi = '0102030405060708'
#key = ''
BUFF = 1024

def pkcs7unpadding(text):
        """
        处理使用PKCS7填充过的数据
        :param text: 解密后的字符串
        :return:
        """
        length = len(text)
        #text=text+" "
        print(text[length-1])
        #unpadding = ord(str(text[length-1]))
        #unpadding=text[length-1]
        #return text[0:length-unpadding]
        return text[0:length-text[length-1]]

def socket_service():
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        # 绑定端口为9001
        s.bind(('localhost', 9001))
        # 设置监听数
        s.listen(10)
    except socket.error as msg:
        print (msg)
        sys.exit(1)
    print ('Waiting connection...')
 
    while 1:
        #调用accept阻塞： 等待请求并接受(程序会停留在这一旦收到连接请求即开启接受数据的线程)
        conn, addr = s.accept()
        # 接收数据
        t = threading.Thread(target=deal_data, args=(conn, addr))
        t.start()

def deal_data(conn, addr):
    #global key
    print ('Accept new connection from {0}'.format(addr))
    # conn.settimeout(500)
    # 收到请求后的回复
    conn.send('Hi, Welcome to the server!'.encode('utf-8'))
     
    key=""
    while 1:
        if not key:
            s1_recv_data = conn.recv(BUFF)
            if s1_recv_data.decode('utf-8') == 'changekey':
                print(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time()))+' start to change key!')
                #使用RSA产生一对公钥和私钥
                (pubkey, privkey) = rsa.newkeys(512, poolsize=8)
                print(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time()))+' create pubkey & privkey')
                #将公钥发送给客户端
                modulus = pubkey.n
                exponent = pubkey.e
                conn.send(str(modulus).encode('utf-8'))
                conn.send(str(exponent).encode('utf-8'))
                
                print(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time()))+' send pubkey')
                #服务端收到消息
                key = conn.recv(BUFF)
                print(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time()))+' recv encrypted aes-key:'+key.decode('utf8','ignore'))
                #服务端用私钥进行解密，得到AES密钥
                key = rsa.decrypt(key, privkey)
                print(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time()))+' decrypt aes-key')
                key = key.decode()
                print(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time()))+' receive Client key:'+key)
                
        # 申请相同大小的空间存放发送过来的文件名与文件大小信息
        fileinfo_size = struct.calcsize('128sl')
        # 接收文件名与文件大小信息
        buf = conn.recv(fileinfo_size)
        # 判断是否接收到文件头信息
        if buf:
            # 获取文件名和文件大小
            filename, filesize = struct.unpack('128sl', buf)
            fn = filename.strip(b'\00')
            fn = fn.decode()
            print ('file new name is {0}, filesize if {1}'.format(str(fn),filesize))
 
            recvd_size = 0  # 定义已接收文件的大小
            # 存储在该脚本所在目录下面
            fp = open('./' + str(fn), 'wb')
            print ('start receiving...')
            
            if not filesize%16==0:
                filesize+=16-filesize%16
            #method=pyDes.des(key,pyDes.ECB,IV,pad=None,padmode=pyDes.PAD_PKCS5)
            # 将分批次传输的二进制流依次写入到文件
            while not recvd_size == filesize:
                
                if filesize - recvd_size > 1024:
                    data = conn.recv(1024)
                    recvd_size += len(data)
                else:
                    #t=filesize - recvd_size
                    #if not t%16==0:
                        #t=t+(16-t%16)
                    data = conn.recv(filesize - recvd_size)
                    #data = conn.recv(1024)
                    #data = conn.recv(t)
                    recvd_size = filesize
                
                print(len(data))
                print(data)
                
                #data = data.decode('utf8','ignore')
                #if(len(data)%3 == 1): 
                    #data += "=="
                #elif(len(data)%3 == 2): 
                    #data += "=" 
                #data=data.encode('utf8')
                #print(data)
                #encodebytes = base64.decodebytes(data)
                #print(encodebytes)
                # 将加密数据转换位bytes类型数据
                cipher = AES.new(key.encode('utf8'), AES.MODE_CBC, vi.encode('utf8'))
                #text_decrypted = cipher.decrypt(encodebytes)
                text_decrypted = cipher.decrypt(data)
                #unpad = lambda s: s[0:-s[-1]]
                #text_decrypted = unpad(text_decrypted)
                print(len(text_decrypted))
                print(text_decrypted)
                if len(text_decrypted)<1024:
                    text_decrypted=pkcs7unpadding(text_decrypted)
                
                #data=method.decrypt(data)
                fp.write(text_decrypted)
            fp.close()
            print ('end receive...')
        # 传输结束断开连接
        conn.close()
        break
        
if __name__ == "__main__":
    socket_service()
    

Waiting connection...
Accept new connection from ('127.0.0.1', 3634)
2022-04-28 19:24:25 start to change key!
2022-04-28 19:24:26 create pubkey & privkey
2022-04-28 19:24:26 send pubkey
2022-04-28 19:24:26 recv encrypted aes-key:U)ȹb'Mqy\&=aQԍM~+sD&u1)!3S
2022-04-28 19:24:26 decrypt aes-key
2022-04-28 19:24:26 receive Client key:6A4B3C7D51145065
file new name is 1.jpg, filesize if 819245
start receiving...
1024
b'\xa9,\x04\xe7L$Xf\x1dY\xf9.uJ>\xa7\xea\xe5\xab\xc8:\x97\xdd\x8bJ\xbf\xf0\xf8\t\xd6\x8b\xe5\xdd}.8\x9b\x9dV\xbf\xff\xa7\xb0bDEmR\xb8\xfb\xc6a\xc8g^D\x1b\xa7\xff5ki7Y\x97\xcb\x00\xae\xa5f\xf6a\xe8_,\t\x0f\xa3+h\xfb\xe0]\xf9\xd9z\'`;\xca\xf4L\xb1\xd4\xa0#}\xce\xa0\xdc\x9b%0\xf4$\x98\xb3c\xcaoR\x8d\x9ct%!\x96(2i\xe5\xf8\x01F\xd3\re\xf0|7P6,\x9c5\xd9\xf1\xc0\xc0\x9f\xad/a\xda\x0c@\xe4\x00\xb3$\xdc\xe9\xd6qn\xd5\xd5\xe2\xbe\xc0\xb0\xb8A\x11\x04\x18\x0e\x86\xc6\xbe\x94&T\x12/\x16\x96\xdbq^\xd8\xe5\x13\xca\xc8LK\xb6>\x98\x94\xd2}\xfcW\xa9\x82\x9cN\xdd\x87\xb5d\x1c\x9f\xa7\x106F\xe

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

